**Task 07: Querying RDF(s)**

In [ ]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 3.2MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N6ea81f4fac9d4fc3b4d9533a4ed5467d (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [ ]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s,p,o)

print("\n")

q1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs":RDFS, "ns":ns}
)


for r in g.query(q1):
  print(r)

http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person


(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [ ]:
for s,p,o in g.triples((None, RDF.type, ns.Person)):  
  print(s)

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    print(s1)

print("\n")

q2 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    {
      ?Subject rdf:type ns:Person.
    } UNION{
      ?subClass rdfs:subClassOf ns:Person.
      ?Subject rdf:type ?subClass   
    }  
  }
  ''',
  initNs = { "rdfs":RDFS, "ns":ns, "rdf":RDF}
)


for r in g.query(q2):
  print(r)


http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [ ]:
for s,p,o in g.triples((None, RDF.type, ns.Person)):  
  for s1,p1,o1 in g.triples((s, None, None)):
    print(s1,p1,o1)

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    for s2,p2,o2 in g.triples((s1, None, None)):
      print(s2,p2,o2)

print("\n")

q2 = prepareQuery('''
  SELECT 
    ?Subject ?Properties ?Class
  WHERE { 
    {
      ?Subject rdf:type ns:Person.
      ?Subject ?Properties ?Class
    } UNION{
      ?subClass rdfs:subClassOf ns:Person.
      ?Subject rdf:type ?subClass.
      ?Subject ?Properties ?Class.   
    }  
  }
  ''',
  initNs = { "rdfs":RDFS, "ns":ns, "rdf":RDF}
)


for r in g.query(q2):
  print(r)

http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://somewher